In [226]:
### Market Basket Analysis & Apriori Algorithm

import pandas as pd
# dataset src: https://www.kaggle.com/datasets/heeraldedhia/groceries-dataset

In [227]:
df=pd.read_csv(r"C:\Users\acer\Downloads\archive (9)\Groceries_dataset.csv")

In [228]:
df.info() # Alright, all columns non-null (at least not NA)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38765 entries, 0 to 38764
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Member_number    38765 non-null  int64 
 1   Date             38765 non-null  object
 2   itemDescription  38765 non-null  object
dtypes: int64(1), object(2)
memory usage: 908.7+ KB


In [229]:
df.dtypes # ok, just can reformat date col

Member_number       int64
Date               object
itemDescription    object
dtype: object

In [230]:
df['Date']=pd.to_datetime(df['Date'],infer_datetime_format=True)
df.dtypes # there we go

Member_number               int64
Date               datetime64[ns]
itemDescription            object
dtype: object

In [231]:
df['itemDescription']=df['itemDescription'].apply(lambda x:x.strip().lower()) # preprocessing of the text

In [232]:
first,last=df['Date'].min(),df['Date'].max()
fy,fm,fd=first.year,first.month,first.day
ly,lm,ld=last.year,last.month,last.day
print(f'Data is collected betweeen {fd}/{fm}/{fy} and {ld}/{lm}/{ly}')

Data is collected betweeen 1/1/2014 and 30/12/2015


In [252]:
df_y=df[df['Date'].dt.year==2015]
#df_14=df

In [253]:
pf_y=df_y.pivot_table(index='Member_number',columns='itemDescription',aggfunc='count').fillna(0)
pf_y

Date                                       \
itemDescription abrasive cleaner artif. sweetener baby cosmetics bags   
Member_number                                                           
1000                         0.0              0.0            0.0  0.0   
1001                         0.0              0.0            0.0  0.0   
1002                         0.0              0.0            0.0  0.0   
1003                         0.0              0.0            0.0  0.0   
1004                         0.0              0.0            0.0  0.0   
...                          ...              ...            ...  ...   
4996                         0.0              0.0            0.0  0.0   
4997                         0.0              0.0            0.0  0.0   
4998                         0.0              0.0            0.0  0.0   
4999                         0.0              0.0            0.0  0.0   
5000                         0.0              0.0            0.0  0.0   

                                                                       \
itemDescription baking powder bathroom cleaner beef berries beverages   
Member_number                                                           
1000                      0.0              0.0  0.0     0.0       0.0   
1001                      0.0              0.0  1.0     0.0       0.0   
1002                      0.0              0.0  0.0     0.0       0.0   
1003                      0.0              0.0  0.0     0.0       0.0   
1004                      0.0              0.0  0.0     0.0       0.0   
...                       ...              ...  ...     ...       ...   
4996                      0.0              0.0  0.0     0.0       0.0   
4997                      0.0              0.0  0.0     0.0       0.0   
4998                      0.0              0.0  0.0     0.0       0.0   
4999                      0.0              0.0  0.0     2.0       0.0   
5000                      0.0              0.0  0.0     0.0       0.0   

                              ...                                              \
itemDescription bottled beer  ... uht-milk vinegar waffles whipped/sour cream   
Member_number                 ...                                               
1000                     0.0  ...      0.0     0.0     0.0                0.0   
1001                     0.0  ...      0.0     0.0     0.0                1.0   
1002                     0.0  ...      0.0     0.0     0.0                0.0   
1003                     0.0  ...      0.0     0.0     0.0                0.0   
1004                     0.0  ...      0.0     0.0     0.0                0.0   
...                      ...  ...      ...     ...     ...                ...   
4996                     1.0  ...      0.0     0.0     0.0                0.0   
4997                     0.0  ...      0.0     0.0     0.0                0.0   
4998                     0.0  ...      0.0     0.0     0.0                0.0   
4999                     0.0  ...      0.0     0.0     0.0                1.0   
5000                     0.0  ...      0.0     0.0     0.0                0.0   

                                                                          
itemDescription whisky white bread white wine whole milk yogurt zwieback  
Member_number                                                             
1000               0.0         0.0        0.0        1.0    1.0      0.0  
1001               0.0         1.0        0.0        0.0    0.0      0.0  
1002               0.0         0.0        0.0        0.0    0.0      0.0  
1003               0.0         0.0        0.0        0.0    0.0      0.0  
1004               0.0         0.0        0.0        0.0    0.0      0.0  
...                ...         ...        ...        ...    ...      ...  
4996               0.0         0.0        0.0        0.0    0.0      0.0  
4997               0.0         0.0        1.0        0.0    0.0      0.0  
4998               0.0

In [254]:
pf_y=pf_y.applymap(lambda x:1 if x>0 else 0) # TIL abt applymap
# so only deals with bools (that's what the 1s and 0s represent) > if bought or nt. Qty doesn't seem to be significant

In [255]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [256]:
pf_y.columns=[col[-1] for col in list(pf_y.columns)]
df_new_y=pd.DataFrame(pf_y)

In [257]:
freq_items = apriori(df_new_y, min_support=0.05, use_colnames=True, max_len=5).sort_values(by='support')
freq_items

c:\Users\acer\AppData\Local\Programs\Python\Python39\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
43,0.050091,"(pip fruit, whole milk)"
45,0.050392,"(rolls/buns, yogurt)"
34,0.050694,"(whole milk, bottled water)"
18,0.051298,(meat)
35,0.051901,"(whole milk, canned beer)"
20,0.052505,(onions)
15,0.054013,(ham)
37,0.057031,"(whole milk, frankfurter)"
42,0.057333,"(yogurt, other vegetables)"
16,0.057333,(hamburger meat)


In [259]:
rules=association_rules(freq_items, metric="confidence", min_threshold=0.3).sort_values('confidence',ascending=False)
rules

# interesting interesting > a lot of antecedents to whole milk being bought
# but nth rlly vice versa

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
12,(yogurt),(whole milk),0.185878,0.352746,0.080567,0.433442,1.228764,0.015000,1.142431,0.228681
1,(bottled water),(whole milk),0.121002,0.352746,0.050694,0.418953,1.187689,0.008011,1.113944,0.179783
13,(rolls/buns),(whole milk),0.226916,0.352746,0.094750,0.417553,1.183722,0.014706,1.111267,0.200764
2,(canned beer),(whole milk),0.125830,0.352746,0.051901,0.412470,1.169312,0.007515,1.101653,0.165638
11,(soda),(whole milk),0.194327,0.352746,0.079059,0.406832,1.153330,0.010510,1.091182,0.165011
3,(frankfurter),(whole milk),0.140314,0.352746,0.057031,0.406452,1.152250,0.007536,1.090482,0.153699
5,(citrus fruit),(whole milk),0.150573,0.352746,0.059747,0.396794,1.124871,0.006632,1.073023,0.130687
10,(sausage),(whole milk),0.197043,0.352746,0.078153,0.396631,1.124410,0.008647,1.072733,0.137796
0,(pip fruit),(whole milk),0.126735,0.352746,0.050091,0.395238,1.120461,0.005385,1.070263,0.123113
14,(other vegetables),(whole milk),0.262824,0.352746,0.102595,0.390356,1.106621,0.009885,1.061692,0.130699
